In [286]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import brightway2 as bw

# Custom utils defined for inter-acv
from utils import *
from expression import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [287]:
# Reset project & DB
bw.projects.set_current('B_Publication')
resetDb()
resetParams()

Db incer-acv was here. Deleting it


In [288]:
alpha = newParamDef(
    'alpha', 
    ParamType.NUMBER, 
    default=0.5, 
    description="alpha")

beta = newParamDef(
    'beta', 
    ParamType.NUMBER, 
    default=0.5, 
    description="beta")

# Define parameters of the model
elect_switch_param = newParamDef(
    'elect_switch_param', 
    ParamType.ENUM, 
    values=["us", "glo"], 
    default="us", 
    description="Switch on electricty mix")

In [289]:
# Select impacts
impacts = [m for m in bw.methods if 'ILCD 1.0.8 2016' in str(m) and 'no LT' in str(m)]

In [290]:
occup = findBio('Occupation, industrial area')
heat = findBio('Heat, waste', categories=['air'])
heat.__dict__

{'_document': <ActivityDataset: 1536>,
 '_data': {'categories': ('air',),
  'code': 'f2d84834-d0b3-42e5-b41a-f04cc80337a4',
  'name': 'Heat, waste',
  'database': 'biosphere3',
  'unit': 'megajoule',
  'type': 'emission'}}

In [291]:
# Create new, parametrized activity

activity1 = newActivity(
    name="act1", unit="kg", 
    exchanges= {
        occup:3, 
        heat: alpha})

activity2 = newActivity(
    name="act2", unit="kg", 
    exchanges= {
        activity1: beta * 5 + alpha, # Nested activity with parametrized amount
        heat:7}) # Constant exhange

In [292]:
print_act(activity1)

,amount /formula,unit,type
"'Occupation, industrial area' (square meter-year, None, ('natural resource', 'land'))",3,square meter-year,biosphere
"'Heat, waste' (megajoule, None, ('air',))",alpha,megajoule,biosphere


In [293]:
print_act(activity2)

,amount /formula,unit,type
"'act1' (kg, GLO, None)",alpha + 5*beta,kg,technosphere
"'Heat, waste' (megajoule, None, ('air',))",7,megajoule,biosphere


In [294]:
elec_switch_activity = switch(
    elect_switch_param, dict(
        us=us_elec, 
        glo=glo_elec))

print_act(elec_switch_activity)

,amount /formula,unit,type
"'market group for electricity, medium voltage' (kilowatt hour, US, None)",elect_switch_param_us,kilowatt hour,technosphere
"'market group for electricity, medium voltage' (kilowatt hour, GLO, None)",elect_switch_param_glo,kilowatt hour,technosphere


In [297]:
computeLCA(activity2, impacts, alpha=4, beta=0.8)

foo


,0
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), resources, mineral, fossils and renewables)",0.000000
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), climate change, GWP 100a)",0.000000
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), ecosystem quality, freshwater and terrestrial acidification)",0.000000
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), ecosystem quality, freshwater ecotoxicity)",0.000000
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), ecosystem quality, freshwater eutrophication)",0.000000
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), ecosystem quality, ionising radiation)",0.000000
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), ecosystem quality, marine eutrophication)",0.000000
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), ecosystem quality, terrestrial eutrophication)",0.000000
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), human health, carcinogenic effects)",0.000000
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), human health, ionising radiation)",0.000000
